In [ ]:
from torchvision.models import resnet18
net = resnet18(num_classes=10).cuda()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18
net = resnet18(num_classes=10).cuda()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]))
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

# Training loop
def train(epoch):
    net.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

# Validation loop
def test():
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    print(f'Accuracy: {100 * correct / total:.2f}%')

# Training and testing
for epoch in range(50):  # number of epochs
    train(epoch)
    test()
    scheduler.step()

print('Finished Training')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18


criterion = nn.CrossEntropyLoss()

# Define the PGD attack
def pgd_attack(model, images, labels, eps, alpha, iters):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()

    ori_images = images.data

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

# Function to test the model on adversarial examples
def test_adversarial(eps):
    net.eval()
    correct = 0
    total = 0
    alpha = eps / 10  # step size
    iters = 10  # number of iterations

    # Generate adversarial examples
    adv_examples = []
    targets_list = []
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, eps, alpha, iters)
        adv_examples.append(adv_inputs)
        targets_list.append(targets)

    # Evaluate the model on adversarial examples
    with torch.no_grad():
        for adv_inputs, targets in zip(adv_examples, targets_list):
            outputs = net(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on adversarial examples with eps={eps}: {accuracy:.2f}%')

# Test the model on adversarial examples with different epsilons
epsilons = [0.25, 1.0, 1.5]
for eps in epsilons:
    test_adversarial(eps)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet18

criterion = nn.CrossEntropyLoss()

# Define the PGD attack with uniform epsilon
def pgd_attack_uniform(model, images, labels, eps, alpha, iters):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()

    ori_images = images.data

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

# Define the PGD attack with spatially varying epsilon
def pgd_attack_spatial(model, images, labels, center_eps, outer_eps, alpha, iters):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()

    ori_images = images.data

    # Create masks for center and outer regions
    center_mask = torch.zeros_like(images)
    center_mask[:, :, 8:24, 8:24] = 1  # Center 16x16 pixels
    outer_mask = 1 - center_mask

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        # Apply different epsilon values for center and outer regions
        adv_images = images + alpha * images.grad.sign()
        center_eta = torch.clamp(adv_images - ori_images, min=-center_eps, max=center_eps) * center_mask
        outer_eta = torch.clamp(adv_images - ori_images, min=-outer_eps, max=outer_eps) * outer_mask
        eta = center_eta + outer_eta
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

    return images

# Function to test the model on adversarial examples
def test_adversarial_uniform(eps):
    net.eval()
    correct = 0
    total = 0
    alpha = eps / 10  # step size
    iters = 10  # number of iterations

    # Generate adversarial examples
    adv_examples = []
    targets_list = []
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack_uniform(net, inputs, targets, eps, alpha, iters)
        adv_examples.append(adv_inputs)
        targets_list.append(targets)

    # Evaluate the model on adversarial examples
    with torch.no_grad():
        for adv_inputs, targets in zip(adv_examples, targets_list):
            outputs = net(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on adversarial examples with eps={eps}: {accuracy:.2f}%')

def test_adversarial_spatial(center_eps, outer_eps):
    net.eval()
    correct = 0
    total = 0
    alpha = outer_eps / 10  # step size based on outer_eps
    iters = 10  # number of iterations

    # Generate adversarial examples
    adv_examples = []
    targets_list = []
    for inputs, targets in testloader:
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack_spatial(net, inputs, targets, center_eps, outer_eps, alpha, iters)
        adv_examples.append(adv_inputs)
        targets_list.append(targets)

    # Evaluate the model on adversarial examples
    with torch.no_grad():
        for adv_inputs, targets in zip(adv_examples, targets_list):
            outputs = net(adv_inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on adversarial examples with center_eps={center_eps} and outer_eps={outer_eps}: {accuracy:.2f}%')

# Test the model on adversarial examples with uniform epsilons
uniform_epsilons = [0.25, 1.0, 1.5]
for eps in uniform_epsilons:
    test_adversarial_uniform(eps)

# Test the model on adversarial examples with spatially varying epsilons
center_eps = 2/255
outer_eps = 8/255
test_adversarial_spatial(center_eps, outer_eps)

In [ ]:
!pip install wget
import wget

In [ ]:
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union

import numpy as np
import torch
import torch.nn as nn
from torch import Tensor
from tqdm import tqdm

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


def conv2x2(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """2x2 convolution"""
    conv = nn.Conv2d(in_planes, out_planes, kernel_size=2, stride=stride, padding=1, bias=False)
    conv.weight = nn.Parameter(torch.ones_like(conv.weight))
    return conv


class preprocess(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        b, c, h, w = input.size()
        output = torch.zeros_like(input).cpu().numpy()

        for i in range(b):
          img = input[i].cpu().numpy()
          mask = np.random.binomial(1, 0.7, h * w).reshape(h, w)
          p_obs = len(mask[mask == 1]) / (h * w)
          for channel in range(c):
              u, sigma, v = np.linalg.svd(img[channel] * mask)
              S = np.zeros((h, w))
              for j in range(int(0.7 * h)):
                  S[j][j] = sigma[j]
              W = np.dot(np.dot(u, S), v) / p_obs
              W[W < -1] = -1
              W[W > 1] = 1
              output[i, channel] = (W + 1) / 2
        output = torch.from_numpy(output).float().to(input.device)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class AliceNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 10,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        self.preprocess = preprocess()

        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.preprocess = preprocess()
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        x = self.preprocess.apply(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        max_inds = x.argmax(dim=1)
        mask = torch.zeros_like(x)
        mask.scatter_(1, max_inds.unsqueeze(1), 1.)
        x *= mask
        x = x / (x + 1e-9)
        return x

    def forward(self, x: Tensor, **kwargs) -> Tensor:
        return self._forward_impl(x)


def get_alicenet() -> AliceNet:

    model = AliceNet(BasicBlock, [2, 2, 2, 2])

    url = 'https://www.dropbox.com/scl/fi/s3vatccoexnyq9p9nr5zo/alice_model_IOAI.pt?rlkey=b6f996keu0vurkoxuubg1tjsm&dl=1'
    wget.download(url, out='./alice_model.pt', bar=None)
    trained_ckpt = torch.load('./alice_model.pt')
    model.load_state_dict(trained_ckpt)

    return model

In [ ]:
alicenet = get_alicenet().cuda()

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from typing import Type, Union, List, Optional, Callable
import wget
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Define the PGD attack with random perturbations
def pgd_attack(model, images, labels, eps, alpha, iters):
    images = images.clone().detach().to(device)
    labels = labels.clone().detach().to(device)
    loss = nn.CrossEntropyLoss()

    ori_images = images.data

    for i in range(iters):
        images.requires_grad = True
        outputs = model(images)

        model.zero_grad()
        cost = loss(outputs, labels).to(device)
        cost.backward()

        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - ori_images, min=-eps, max=eps)
        images = torch.clamp(ori_images + eta, min=0, max=1).detach_()

        # Add random perturbations
        images = images + (torch.rand_like(images) - 0.5) * 2 * eps

    return images

# Function to test the model on adversarial examples
def test_adversarial(model, eps):
    model.eval()
    correct = 0
    total = 0
    alpha = eps / 10  # step size
    iters = 100  # number of iterations

    for inputs, targets in tqdm(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(model, inputs, targets, eps, alpha, iters)
        outputs = model(adv_inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on adversarial examples with eps={eps}: {accuracy:.2f}%')

# Test the model on adversarial examples with different epsilons
epsilons = [0.25,1,1.5]
for eps in epsilons:
    test_adversarial(alicenet, eps)